In [1]:
# Import libraries
import numpy as np
import geopandas as gpd
import momepy
import networkx as nx
# import pandas as pd
# import shapely
# import shapely.geometry as sg
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline

from lmzintgraf_gp_pref_elicit import dataset, gaussian_process, acquisition_function
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_ccs as utils_ccs
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_data as utils_data
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_experiment as utils_experiment
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_parameters as utils_parameters
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_user as utils_user

In [2]:
map = gpd.read_file("Sidewalk_width_crossings_small.geojson") #Read in the map with radius 250m and ~1000 nodes

# Objectives
objective1 = map['length']
objective2 = map['crossing']
objective3 = map['obstacle_free_width']

objectives = ('length', 'crossings')

In [3]:
# Create a NetworkX graph from the map
G = momepy.gdf_to_nx(map, approach='primal')
nodes = G.nodes
edges = G.edges

/usr/local/anaconda3/envs/test/lib/python3.10/site-packages/momepy/utils.py:252: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_network[length] = gdf_network.geometry.length


In [4]:
# print(nodes)

In [5]:
#Pick random ones or pick manually that make sense - to experiment
S = (120548.6120283842, 486088.19577846595)
T = (120798.0893320718, 486128.7633437495)

In [6]:
# Initialise the Gaussian process for 2 objectives
gp = gaussian_process.GPPairwise(num_objectives=2, std_noise=0.01, kernel_width=0.15,prior_mean_type='zero', seed=None)

In [7]:
P = [] #Pareto set
p = []

# Path initialisation
for i in objectives:
    p = nx.shortest_path(G, source=S, target=T, weight=i, method='dijkstra') #Dijkstra algorithm
    P.append(p)

In [8]:
C = min(P, key=lambda p: (p[0], p[1]))
C

[(120548.6120283842, 486088.19577846595),
 (120558.58272730978, 486088.5913559315),
 (120562.22249291666, 486088.4604878346),
 (120597.87366172913, 486118.3885286597),
 (120603.94807229919, 486112.8244804511),
 (120610.1211584085, 486110.4734641692),
 (120615.236697444, 486109.4981894336),
 (120622.18953041689, 486108.8516084536),
 (120627.6055477486, 486108.80491815176),
 (120637.3711853829, 486107.4803808584),
 (120638.32762784461, 486107.6578267999),
 (120644.82334925648, 486107.55756407854),
 (120646.58367929762, 486107.4539825946),
 (120653.63177241654, 486106.9806947992),
 (120658.16234661153, 486106.63926980837),
 (120664.93105615664, 486106.10542439454),
 (120671.8192466126, 486105.5397502568),
 (120678.46251254133, 486104.92881432245),
 (120683.05468505199, 486104.3662392544),
 (120689.70658339545, 486103.4113373837),
 (120697.63022968677, 486102.02209520293),
 (120704.74846380341, 486100.38072378177),
 (120714.24315622302, 486097.30355140124),
 (120715.2451666701, 486096.9044

In [9]:
# User ranking: Compare paths in P
user_preference = utils_user.UserPreference(num_objectives=2, std_noise=0.1)

In [10]:
#TODO: This compiles for eternity. Idk why :'(
# user_preference.rescale_on_input_domain(C)

In [11]:
#TODO: This gives an error from x=x[np.argsort(-utility)] from utils_user.py. So I used user_preference.get_preference (aka the next jupyter code cell, but idk if it's the correct method.
#user_ranking = user_preference.ranking(C, add_noise=True)

In [12]:
add_noise = True
preference = user_preference.get_preference(C,add_noise=add_noise)
print(preference)

[5816.33936573 5816.6072482  5816.58258734 5816.56337878 5816.50851088
 5816.72682005 5816.69805017 5816.73683684 5816.47181699 5816.49052616
 5816.60378345 5816.51549633 5816.59487399 5816.30489073 5816.57950735
 5816.50538176 5816.77003947 5816.48617236 5816.52968209 5816.58239763
 5816.72665978 5816.29770078 5816.57482681 5816.55043948 5816.6539278
 5816.70620801 5816.62290061 5816.61053756 5816.40377817 5816.66319192
 5816.55924865 5816.52885955 5816.39879918 5816.4833986  5816.386183
 5816.6039002  5816.5712521  5816.2865066  5816.48224076 5816.50211564
 5816.35584837 5816.54730653 5816.62265249 5816.51595654 5816.54836534
 5816.72539326 5816.43566629 5816.37191595 5816.55379066 5816.47136512
 5816.68915277 5816.4012239  5816.50430511 5816.59970101 5816.49359338
 5816.53171224 5816.65154896 5816.62195724 5816.64772174 5816.4215806
 5816.51073673 5816.72053474 5816.76912461 5816.56720497 5816.52665913
 5816.56828342 5816.58353105 5816.67492945 5816.5076461  5816.6372081
 5816.61610

/Users/alisatodorova/Desktop/Thesis/Interactive-MOPPA/lmzintgraf_gp_pref_elicit/gp_utilities/utils_user.py:254: RuntimeWarning: overflow encountered in exp
  y += 1. / (1 + np.exp(- x * (a-i) + (b+i)))


In [13]:
# Add the comparisons to GP
comparisons = dataset.DatasetPairwise(num_objectives=2)
preference_copy = np.copy(preference)
comparisons.add_ranked_preferences(preference_copy)
gp.update(comparisons)

In [14]:
#TODO: This code is working but I don't know for what to use it...
# params = utils_parameters.get_parameter_dict(query_type='ranking', utility_noise=0.01, num_objectives=2)
# params['num queries'] = 10
# params['seed'] = 123
#
# # Run the experiment
# experiment = utils_experiment.Experiment(params)
#
# result = experiment.run(recalculate=False)
#
#  # -- true utility --
#
# # get the input domain
# input_domain = result[1]
# idx_order = np.argsort(-input_domain[:, 0])
#
# # get the true utility function
# true_utility = result[2]
# print(true_utility)

In [15]:
# Find the path the user likes best and has the maximum a posteriori (MAP) estimate
gp_sample = gp.sample(p)
print(gp_sample)

[-0.95724841  0.09655672 -0.4048892   0.03389495  0.76182351 -0.26825858
 -0.15294788  0.41891858 -1.3470107  -0.63591126 -0.33571447  0.56281867
 -1.72477563 -1.44099637 -0.95837681 -0.3214679  -0.30798345 -0.44959541
  0.42494426  1.04182353 -0.62498868  0.01115805  0.46857372 -0.38273271
  0.50690071 -0.81855324 -1.17490486  0.86333277 -0.50581387 -1.2166946
 -0.19491264 -1.2479632   0.35274965 -0.03731987  0.80028     0.20634282
 -0.37899376 -0.06284924  0.37225561 -0.42057649 -1.22620193  0.56680111
 -0.09841915 -1.06731848 -0.59476126  0.62606403  0.1507684   0.85123235
  0.43773078 -0.89365022  0.06632805 -0.7856956  -0.10570271 -0.18662513
  0.19291645 -0.2650385  -0.16689673  0.536486   -0.93390565 -1.63903312
  0.86638151  0.39543652  1.24332149 -0.28426692  0.25846068  1.03076848
 -0.18311858  0.1502338  -1.24741479 -0.67966185  0.48918709 -0.17686172
  1.22762433 -0.00246232 -0.25710982 -0.01477593 -0.68104517  0.47566448
 -1.44042631 -0.35638735  0.59460335]


In [16]:
p_star_index = np.argmax(gp_sample)
p_star_index

62

In [17]:
p_star = p[p_star_index]
p_star

(120923.52853601528, 486037.62249296077)

In [18]:
#TODO: No idea if this is correct. I'm not sure what ccs_size should be. So I put 10, as well as for min_size, in order to run it quicker. Otherwise, it takes too much time and gets stuck.
input_domain = utils_ccs.get_pcs_grid(ccs_size=10, num_objectives=2, eucledian_dist=0.05, min_size=10, seed=123)
#Set eucledian_dist smaller to run the code faster
#Setting ccs_size and min_size to the same value is probably also a good idea.
input_domain

array([[0.96437258, 0.23727299],
       [0.63115621, 0.85134756],
       [0.99846441, 0.0564509 ],
       [0.11505206, 0.99825729],
       [0.94036015, 0.3278353 ],
       [0.        , 1.        ],
       [0.91954989, 0.40080325],
       [0.26688587, 0.95992014],
       [1.        , 0.        ],
       [0.76035816, 0.7034629 ],
       [0.03835069, 0.9994191 ],
       [0.07670137, 0.99883819],
       [0.15301051, 0.988673  ],
       [0.19096897, 0.97908871],
       [0.22892742, 0.96950442],
       [0.31241967, 0.94634856],
       [0.35795346, 0.93277699],
       [0.40348725, 0.91920542],
       [0.44902104, 0.90563385],
       [0.49455483, 0.89206227],
       [0.54008863, 0.8784907 ],
       [0.58562242, 0.86491913],
       [0.67422352, 0.80205267],
       [0.71729084, 0.75275779],
       [0.78309983, 0.66022581],
       [0.80584151, 0.61698872],
       [0.82858319, 0.57375162],
       [0.85132486, 0.53051453],
       [0.87406654, 0.48727744],
       [0.89680822, 0.44404035],
       [0.

In [19]:
# Initialise the acquisition function
acq_fun = acquisition_function.DiscreteAcquirer(input_domain=input_domain, query_type='ranking', seed=123, acquisition_type='expected improvement')

In [20]:
#TODO: The next code cells are in a while-loop
# while C:

In [21]:
expected_improvement = acquisition_function.get_expected_improvement(input_domain, gp, acq_fun.history)
# acq_fun_start_points = acq_fun.get_next_point_EI(gp_sample, C)

In [22]:
t_index = np.argmax(expected_improvement)
t_index

0

In [23]:
t = input_domain[t_index]
t

array([0.96437258, 0.23727299])

In [24]:
C

[(120548.6120283842, 486088.19577846595),
 (120558.58272730978, 486088.5913559315),
 (120562.22249291666, 486088.4604878346),
 (120597.87366172913, 486118.3885286597),
 (120603.94807229919, 486112.8244804511),
 (120610.1211584085, 486110.4734641692),
 (120615.236697444, 486109.4981894336),
 (120622.18953041689, 486108.8516084536),
 (120627.6055477486, 486108.80491815176),
 (120637.3711853829, 486107.4803808584),
 (120638.32762784461, 486107.6578267999),
 (120644.82334925648, 486107.55756407854),
 (120646.58367929762, 486107.4539825946),
 (120653.63177241654, 486106.9806947992),
 (120658.16234661153, 486106.63926980837),
 (120664.93105615664, 486106.10542439454),
 (120671.8192466126, 486105.5397502568),
 (120678.46251254133, 486104.92881432245),
 (120683.05468505199, 486104.3662392544),
 (120689.70658339545, 486103.4113373837),
 (120697.63022968677, 486102.02209520293),
 (120704.74846380341, 486100.38072378177),
 (120714.24315622302, 486097.30355140124),
 (120715.2451666701, 486096.9044

In [25]:
# Remove t from C
C = np.delete(C, np.where(np.all(C == t)))
C

array([120548.61202838, 486088.19577847, 120558.58272731, 486088.59135593,
       120562.22249292, 486088.46048783, 120597.87366173, 486118.38852866,
       120603.9480723 , 486112.82448045, 120610.12115841, 486110.47346417,
       120615.23669744, 486109.49818943, 120622.18953042, 486108.85160845,
       120627.60554775, 486108.80491815, 120637.37118538, 486107.48038086,
       120638.32762784, 486107.6578268 , 120644.82334926, 486107.55756408,
       120646.5836793 , 486107.45398259, 120653.63177242, 486106.9806948 ,
       120658.16234661, 486106.63926981, 120664.93105616, 486106.10542439,
       120671.81924661, 486105.53975026, 120678.46251254, 486104.92881432,
       120683.05468505, 486104.36623925, 120689.7065834 , 486103.41133738,
       120697.63022969, 486102.0220952 , 120704.7484638 , 486100.38072378,
       120714.24315622, 486097.3035514 , 120715.24516667, 486096.90441746,
       120723.45147425, 486093.31371893, 120732.28222387, 486088.65861339,
       120740.35952745, 4

In [26]:
# Inner-loop
# path = A(t, G, S, T)

In [27]:
#TODO: Line 15 of pseudocode is unclear to me how it should be in code. Also, which is correct:
# If v^p improves in the target region
# because you've identified a new value vector on the PCS. If you stop once the utility no longer improves, I think this can result in stopping prematurely. Specifically, imagine you have a current partial Pareto front of (10,0) and (0,10) the user model u((10,0)) is the current best. The target vector is (10,10)  and when you run DFS, you get one of the possible vectors in the target region. You get (1,9) out of the the call to DFS, and the u((1,9)) < u((10,0)) even after querying the user about it. Now here, you shouldn't stop, because the true best - (7,3) for example, is still possible to find.
#you're just not going to improve on that with a newly found vector
#So improving on the acquisition function by identifying a new point is impossible as
# 1) you were searching at an optimistic estimate (target), so the actually found value will be worse than the target
# 2) finding new points, and querying the user reduces uncertainty

In [28]:
P = P.append(p)

In [29]:
#TODO: More user ranking...
#Compare p to p∗ and add comparison to the GP ▷User ranking, i.e., is the new path preferred to the current, maximum one?


In [30]:
# if u(p) > u(p∗) then
# p∗ ← p
# end if


In [31]:
# Compute new candidate targets based on v^p and add to C #p from inner-loop I assume


In [32]:
# end if
# end while
# return p∗, vp
